In [1]:
import os 
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

import json
from dotenv import load_dotenv

# API 암호화를 위해 dotenv를 활용 
load_dotenv()
huggingface_token = os.environ.get("HUGGINGFACE_TOKEN")

# 데이터 불러오기 

In [2]:
# 시스템 프롬프트 설정 
system_message = "당신은 이미지와 제품명(name)으로부터 패션/스타일 정보를 추론하는 분류 모델입니다."

# 실제로 사용자 입력 -> 모델이 답해야 하는 프롬프트
prompt = """입력 정보:
- name: {name}
- image: [image]

위 정보를 바탕으로, 아래 7가지 key에 대한 값을 JSON 형태로 추론해 주세요:
1) gender
2) masterCategory
3) subCategory
4) season
5) usage
6) baseColour
7) articleType

출력 시 **아래 JSON 예시 형태**를 반드시 지키세요:
{{
  "gender": "예시값",
  "masterCategory": "예시값",
  "subCategory": "예시값",
  "season": "예시값",
  "usage": "예시값",
  "baseColour": "예시값",
  "articleType": "예시값"
}}

# 예시
{{
  "gender": "Men",
  "masterCategory": "Accessories",
  "subCategory": "Eyewear",
  "season": "Winter",
  "usage": "Casual",
  "baseColour": "Blue",
  "articleType": "Sunglasses"
}}

# 주의
- 7개 항목 이외의 정보(텍스트, 문장 등)는 절대 포함하지 마세요.
"""

In [7]:
from datasets import load_dataset

# label을 생성하기 위한 함수 
def combine_cols_to_label(example):
    label_dict = {
        "gender": example["gender"],
        "masterCategory": example["masterCategory"],
        "subCategory": example["subCategory"],
        "season": example["season"],
        "usage": example["usage"],
        "baseColour": example["baseColour"],
        "articleType": example["articleType"],
    }
    example["label"] = json.dumps(label_dict, ensure_ascii=False)
    return example

# 대화용 포맷으로 변환하는 함수 
def format_data(sample):
    return {
        "messages": [
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": system_message
                    }
                ],
            },
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        # 제품명 
                        "text": prompt.format(name=sample["productDisplayName"]),
                    },
                    {
                        "type": "image",
                        # 이미지 파일 
                        "image": sample["file_path"],  
                    }
                ],
            },
            {
                "role": "assistant",
                "content": [
                    {
                        "type": "text",
                        # combine_cols_to_label에서 만든 JSON 문자열
                        "text": sample["label"],
                    }
                ],
            },
        ],
    }

# 전처리한 데이터셋 불러오기 
dataset = load_dataset("daje/kaggle-image-datasets", split="train")
dataset_add_label = dataset.map(combine_cols_to_label)
dataset_add_label = dataset_add_label.shuffle(seed=4242)

README.md:   0%|          | 0.00/665 [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/221M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/44440 [00:00<?, ? examples/s]

Map:   0%|          | 0/44440 [00:00<?, ? examples/s]

In [8]:
# 라벨을 추가한 데이터의 예시 
dataset_add_label[0]

{'file_path': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=60x80>,
 'id': 44429,
 'gender': 'Men',
 'masterCategory': 'Footwear',
 'subCategory': 'Shoes',
 'articleType': 'Formal Shoes',
 'baseColour': 'Black',
 'season': 'Summer',
 'year': '2013',
 'usage': 'Formal',
 'productDisplayName': 'Gliders Men Black Formal Shoes',
 'label': '{"gender": "Men", "masterCategory": "Footwear", "subCategory": "Shoes", "season": "Summer", "usage": "Formal", "baseColour": "Black", "articleType": "Formal Shoes"}'}

In [9]:
# 메세지 포맷으로 데이터셋 변환 
formatted_dataset = [format_data(row) for row in dataset_add_label]

In [10]:
# 메세지 포맷으로 변환 데이터 예시 
formatted_dataset[0]

{'messages': [{'role': 'system',
   'content': [{'type': 'text',
     'text': '당신은 이미지와 제품명(name)으로부터 패션/스타일 정보를 추론하는 분류 모델입니다.'}]},
  {'role': 'user',
   'content': [{'type': 'text',
     'text': '입력 정보:\n- name: Gliders Men Black Formal Shoes\n- image: [image]\n\n위 정보를 바탕으로, 아래 7가지 key에 대한 값을 JSON 형태로 추론해 주세요:\n1) gender\n2) masterCategory\n3) subCategory\n4) season\n5) usage\n6) baseColour\n7) articleType\n\n출력 시 **아래 JSON 예시 형태**를 반드시 지키세요:\n{\n  "gender": "예시값",\n  "masterCategory": "예시값",\n  "subCategory": "예시값",\n  "season": "예시값",\n  "usage": "예시값",\n  "baseColour": "예시값",\n  "articleType": "예시값"\n}\n\n# 예시\n{\n  "gender": "Men",\n  "masterCategory": "Accessories",\n  "subCategory": "Eyewear",\n  "season": "Winter",\n  "usage": "Casual",\n  "baseColour": "Blue",\n  "articleType": "Sunglasses"\n}\n\n# 주의\n- 7개 항목 이외의 정보(텍스트, 문장 등)는 절대 포함하지 마세요.\n'},
    {'type': 'image',
     'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=60x80>}]},
  {'role': 'assistant',
   'con

In [11]:
from sklearn.model_selection import train_test_split

# test_size=0.1로 설정하여 전체 데이터의 10%를 테스트 세트로 분리
train_dataset, test_dataset = train_test_split(formatted_dataset, 
                                             test_size=0.1, 
                                             random_state=42)

In [12]:
# train_dataset, test_dataset 데이터 수 확인 
len(train_dataset), len(test_dataset)

(39996, 4444)

In [13]:
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor, BitsAndBytesConfig

# 허깅페이스 모델 ID
model_id = "Qwen/Qwen2-VL-7B-Instruct" 

# 모델과 프로세서 로드
model = AutoModelForVision2Seq.from_pretrained(
   model_id,
   device_map="auto",                            # GPU 메모리에 자동 할당
   torch_dtype=torch.bfloat16,                   # bfloat16 정밀도 사용
)
processor = AutoProcessor.from_pretrained(model_id)  # 텍스트/이미지 전처리기 로드

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/56.5k [00:00<?, ?B/s]

model-00001-of-00005.safetensors:   0%|          | 0.00/3.90G [00:00<?, ?B/s]

model-00002-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

model-00003-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [14]:
# Preparation for inference
text = processor.apply_chat_template(
    train_dataset[2]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|im_start|>system
당신은 이미지와 제품명(name)으로부터 패션/스타일 정보를 추론하는 분류 모델입니다.<|im_end|>
<|im_start|>user
입력 정보:
- name: Myntra Women's I Want You Black T-shirt
- image: [image]

위 정보를 바탕으로, 아래 7가지 key에 대한 값을 JSON 형태로 추론해 주세요:
1) gender
2) masterCategory
3) subCategory
4) season
5) usage
6) baseColour
7) articleType

출력 시 **아래 JSON 예시 형태**를 반드시 지키세요:
{
  "gender": "예시값",
  "masterCategory": "예시값",
  "subCategory": "예시값",
  "season": "예시값",
  "usage": "예시값",
  "baseColour": "예시값",
  "articleType": "예시값"
}

# 예시
{
  "gender": "Men",
  "masterCategory": "Accessories",
  "subCategory": "Eyewear",
  "season": "Winter",
  "usage": "Casual",
  "baseColour": "Blue",
  "articleType": "Sunglasses"
}

# 주의
- 7개 항목 이외의 정보(텍스트, 문장 등)는 절대 포함하지 마세요.
<|vision_start|><|image_pad|><|vision_end|><|im_end|>
<|im_start|>assistant
{"gender": "Women", "masterCategory": "Apparel", "subCategory": "Topwear", "season": "Summer", "usage": "Casual", "baseColour": "Black", "articleType": "Tshirts"}<|im_end|>



In [15]:
import torch
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from qwen_vl_utils import process_vision_info

#############################################
# "배치 추론"용 함수
#############################################
def generate_batch_description(batch_messages, model, processor):
    # 배치 전체의 text / images / videos를 한번에 준비
    texts = []
    all_image_inputs = []
    all_video_inputs = []

    # (A) 각 샘플마다 QWen-VL용 텍스트 생성 + 이미지/비디오 추출
    for messages in batch_messages:
        # 1) QWen-VL 텍스트 템플릿 생성
        text_prompt = processor.apply_chat_template(
            messages, 
            tokenize=False, 
            add_generation_prompt=True
        )
        texts.append(text_prompt)

        # 2) 이미지/비디오 추출
        image_inputs, video_inputs = process_vision_info(messages)

        # 비디오가 없는 경우가 대부분이면, video_inputs를 무조건 None으로 처리해도 됨.
        all_image_inputs.append(image_inputs[0] if image_inputs else None)
        all_video_inputs.append(video_inputs[0] if video_inputs else None)

    # (B) 비디오가 전혀 없으면 videos=None으로 넘기도록 처리
    if any(x is not None for x in all_video_inputs):
        videos_to_pass = all_video_inputs
    else:
        videos_to_pass = None  

    # (C) processor로 배치 전체 인코딩
    inputs = processor(
        text=texts,
        images=all_image_inputs if any(x is not None for x in all_image_inputs) else None,
        videos=videos_to_pass,
        return_tensors="pt",
        padding=True
    )
    inputs = inputs.to(model.device)

    # (D) 모델 추론
    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_new_tokens=128,
            top_p=1.0,
            do_sample=True,
            temperature=0.1
        )

    # (E) 디코딩
    batch_outputs = []
    for i, out_ids in enumerate(generated_ids):
        prompt_len = len(inputs.input_ids[i])
        trimmed_out_ids = out_ids[prompt_len:]
        decoded = processor.decode(
            trimmed_out_ids,
            skip_special_tokens=True,
            clean_up_tokenization_spaces=False
        )
        batch_outputs.append(decoded)

    return batch_outputs

In [16]:
model.eval()
before_results = []
batch_size = 128

# 배치단위로 for문을 돌면서 generate_batch_description함수를 이용하여 인퍼런스를 진행합니다. 
for start_idx in tqdm(range(0, len(test_dataset), batch_size)):
    batch_data = test_dataset[start_idx : start_idx + batch_size]

    batch_messages = []
    answers = []  # 각 샘플 정답
    for item in batch_data:
        # system+user 메시지
        sys_usr = item["messages"][:2]
        batch_messages.append(sys_usr)

        # 정답(assistant)
        ans_text = item["messages"][2]["content"][0]["text"]
        answers.append(ans_text)

    # 한 번에 모델 추론
    predicted_texts = generate_batch_description(batch_messages, model, processor)

    # 결과 저장
    for ans, pred in zip(answers, predicted_texts):
        before_results.append((ans, pred))

  0%|          | 0/35 [00:00<?, ?it/s]

In [18]:
before_results[0]

('{"gender": "Women", "masterCategory": "Accessories", "subCategory": "Jewellery", "season": "Fall", "usage": "Casual", "baseColour": "Black", "articleType": "Necklace and Chains"}',
 '{\n  "gender": "Women",\n  "masterCategory": "Accessories",\n  "subCategory": "Jewelry",\n  "season": "Any",\n  "usage": "Casual",\n  "baseColour": "Black",\n  "articleType": "Necklace"\n}')

In [19]:
import json_repair
print(json_repair.repair_json(before_results[0][0]))
print(json_repair.repair_json(before_results[0][1]))

{"gender": "Women", "masterCategory": "Accessories", "subCategory": "Jewellery", "season": "Fall", "usage": "Casual", "baseColour": "Black", "articleType": "Necklace and Chains"}
{"gender": "Women", "masterCategory": "Accessories", "subCategory": "Jewelry", "season": "Any", "usage": "Casual", "baseColour": "Black", "articleType": "Necklace"}


In [20]:
json_repair.repair_json(before_results[0][0]) == json_repair.repair_json(before_results[0][1])

False

In [21]:
# Accuracy 계산

correct_count = sum(1 for (ans, pred) in before_results if json_repair.repair_json(ans) == json_repair.repair_json(pred))
accuracy = correct_count / len(before_results) * 100
print(f"Accuracy: {accuracy:.2f}%")

Accuracy: 0.25%


In [22]:
before_json_results = [(json_repair.repair_json(ans, return_objects=True), json_repair.repair_json(pred, return_objects=True)) for (ans, pred) in before_results]
before_answers = [ans for (ans, pred) in before_json_results]
before_predicts = [pred for (ans, pred) in before_json_results]

In [23]:
before_answers[0], before_predicts[0]

({'gender': 'Women',
  'masterCategory': 'Accessories',
  'subCategory': 'Jewellery',
  'season': 'Fall',
  'usage': 'Casual',
  'baseColour': 'Black',
  'articleType': 'Necklace and Chains'},
 {'gender': 'Women',
  'masterCategory': 'Accessories',
  'subCategory': 'Jewelry',
  'season': 'Any',
  'usage': 'Casual',
  'baseColour': 'Black',
  'articleType': 'Necklace'})

In [24]:
# (Micro) 방식으로 전체 필드를 한꺼번에 세어 정확도(Accuracy) 구하는 함수 
def field_accuracy_micro(y_true_list, y_pred_list):

    total_matched = 0
    total_fields = 0
    
    for dtrue, dpred in zip(y_true_list, y_pred_list):
        for key in dtrue.keys():
            total_fields += 1
            if key in dpred and dtrue[key] == dpred[key]:
                total_matched += 1
    
    # 분모가 0인 경우(비어 있는 dict 등)를 대비
    return total_matched / total_fields if total_fields > 0 else 0

# (Macro) 방식으로 각 샘플별 Accuracy를 구한 뒤 평균 구하는 함수 
def field_accuracy_macro(y_true_list, y_pred_list):
    
    acc_list = []
    for dtrue, dpred in zip(y_true_list, y_pred_list):
        matched = 0
        total = len(dtrue.keys())  
        for key in dtrue.keys():
            if key in dpred and dtrue[key] == dpred[key]:
                matched += 1
        
        acc = matched / total if total > 0 else 0
        acc_list.append(acc)
    
    # 모든 샘플에 대한 Accuracy의 평균
    return sum(acc_list) / len(acc_list) if len(acc_list) > 0 else 0


# 전체 필드를 한꺼번에 세는 방식 (Micro)
print(f"Field Accuracy (Micro):{field_accuracy_micro(before_answers, before_predicts)*100:.2f}%")

# 샘플별 Accuracy를 구한 뒤 평균 (Macro)
print(f"Field Accuracy (Macro):{field_accuracy_macro(before_answers, before_predicts)*100:.2f}%")


Field Accuracy (Micro):50.32%
Field Accuracy (Macro):50.32%


In [25]:
# precision, recall, f1 구하는 함수 
## (Micro) 방식으로 전체 필드를 한꺼번에 세어 정확도(Accuracy)를 구하는 함수 
def dict_set_scores_micro(y_true_list, y_pred_list):

    total_true_pairs = 0
    total_pred_pairs = 0
    total_intersect_pairs = 0
    
    for dtrue, dpred in zip(y_true_list, y_pred_list):
        # 1) 만약 빈 문자열 "" 이거나 None이라면, 빈 dict로 처리
        if isinstance(dpred, str) and dpred.strip() == "":
            dpred = {}
        if dpred is None:
            dpred = {}
        
        # 필요하다면 y_true도 동일하게 처리 (빈 문자열일 수 있다면)
        if isinstance(dtrue, str) and dtrue.strip() == "":
            dtrue = {}
        if dtrue is None:
            dtrue = {}
        
        # 2) (key, value) 쌍 집합 만들기
        true_pairs = set(dtrue.items())
        pred_pairs = set(dpred.items())
        
        intersection = true_pairs.intersection(pred_pairs)
        
        total_true_pairs += len(true_pairs)
        total_pred_pairs += len(pred_pairs)
        total_intersect_pairs += len(intersection)
    
    precision = total_intersect_pairs / total_pred_pairs if total_pred_pairs else 0
    recall = total_intersect_pairs / total_true_pairs if total_true_pairs else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) else 0
    
    return {
        "precision": precision,
        "recall": recall,
        "f1": f1
    }

scores_micro = dict_set_scores_micro(before_answers, before_predicts)
print(f'Micro Scores(Precision) : {scores_micro["precision"]*100:.2f}%')
print(f'Micro Scores(Recall) : {scores_micro["recall"]*100:.2f}%')
print(f'Micro Scores(F1)  {scores_micro["f1"]*100:.2f}%')


## 샘플별로 (k,v) 쌍 집합 Precision, Recall, F1을 구하고, 그걸 다시 평균(Macro) 구하는 함수 
def dict_set_scores_macro(y_true_list, y_pred_list):
    
    precisions = []
    recalls = []
    f1s = []
    
    for dtrue, dpred in zip(y_true_list, y_pred_list):
        # 1) 만약 빈 문자열 "" 이거나 None이라면, 빈 dict로 처리
        if isinstance(dpred, str) and dpred.strip() == "":
            dpred = {}
        if dpred is None:
            dpred = {}
        
        # 필요하다면 y_true도 동일하게 처리 (빈 문자열일 수 있다면)
        if isinstance(dtrue, str) and dtrue.strip() == "":
            dtrue = {}
        if dtrue is None:
            dtrue = {}
        
        # 2) (key, value) 쌍 집합 만들기
        true_pairs = set(dtrue.items())
        pred_pairs = set(dpred.items())
        
        # 3) 교집합 크기
        intersection = true_pairs.intersection(pred_pairs)
        
        # 4) Precision, Recall, F1
        precision = len(intersection) / len(pred_pairs) if len(pred_pairs) > 0 else 0
        recall = len(intersection) / len(true_pairs) if len(true_pairs) > 0 else 0
        f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
        
        # 5) 리스트에 저장
        precisions.append(precision)
        recalls.append(recall)
        f1s.append(f1)
    
    # 6) 모든 샘플의 평균(Macro)
    n = len(precisions)  # = len(y_true_list) = len(y_pred_list)
    avg_precision = sum(precisions) / n if n > 0 else 0
    avg_recall = sum(recalls) / n if n > 0 else 0
    avg_f1 = sum(f1s) / n if n > 0 else 0
    
    return {
        "precision": avg_precision,
        "recall": avg_recall,
        "f1": avg_f1
    }

scores_macro = dict_set_scores_macro(before_answers, before_predicts)
print(f'Macro Scores(Precision) : {scores_macro["precision"]*100:.2f}%')
print(f'Macro Scores(Recall) : {scores_macro["recall"]*100:.2f}%')
print(f'Macro Scores(F1)  {scores_macro["f1"]*100:.2f}%')

Micro Scores(Precision) : 51.57%
Micro Scores(Recall) : 50.32%
Micro Scores(F1)  50.94%
Macro Scores(Precision) : 50.32%
Macro Scores(Recall) : 50.32%
Macro Scores(F1)  50.32%


In [26]:
def to_dict_if_str(obj):
    """
    obj가 str이면 JSON으로 로드하고,
    이미 dict 등의 타입이면 그대로 반환.
    """
    if isinstance(obj, str):
        try:
            return json.loads(obj)
        except json.JSONDecodeError:
            # json.loads()가 실패하면 json_repair로 한 번 더 시도
            return json_repair.repair_json(obj, return_objects=True)
    return obj

def compute_field_accuracy(predictions, targets, field):
    total = 0
    correct = 0
    
    for pred, target in zip(predictions, targets):
        # 문자열(JSON)인지 확인 후 dict로 변환
        pred_dict = to_dict_if_str(pred)
        target_dict = to_dict_if_str(target)
        
        if not isinstance(pred_dict, dict) or not isinstance(target_dict, dict):
            continue
        
        if field in pred_dict and field in target_dict:
            total += 1
            if pred_dict[field] == target_dict[field]:
                correct += 1
    
    return correct / total if total > 0 else 0

print("----")

import json_repair
before_json_results = [(json_repair.repair_json(ans, return_objects=True), json_repair.repair_json(pred, return_objects=True)) for (ans, pred) in before_results]
before_answers = [ans for (ans, pred) in before_json_results]
before_predicts = [pred for (ans, pred) in before_json_results]

# 특정 필드별 정확도
gender_accuracy = compute_field_accuracy(before_predicts, before_answers, 'gender')
print(f"Gender accuracy: {gender_accuracy:.2%}")
masterCategory_accuracy = compute_field_accuracy(before_predicts, before_answers, 'masterCategory')
print(f"masterCategory accuracy: {masterCategory_accuracy:.2%}")
subCategory_accuracy = compute_field_accuracy(before_predicts, before_answers, 'subCategory')
print(f"subCategory accuracy: {subCategory_accuracy:.2%}")
articleType_accuracy = compute_field_accuracy(before_predicts, before_answers, 'articleType')
print(f"articleType accuracy: {articleType_accuracy:.2%}")
season_accuracy = compute_field_accuracy(before_predicts, before_answers, 'season')
print(f"season accuracy: {season_accuracy:.2%}")
usage_accuracy = compute_field_accuracy(before_predicts, before_answers, 'usage')
print(f"usage accuracy: {usage_accuracy:.2%}")
baseColour_accuracy = compute_field_accuracy(before_predicts, before_answers, 'baseColour')
print(f"baseColour accuracy: {baseColour_accuracy:.2%}")

----
Gender accuracy: 97.00%
masterCategory accuracy: 37.58%
subCategory accuracy: 16.88%
articleType accuracy: 19.35%
season accuracy: 30.02%
usage accuracy: 76.87%
baseColour accuracy: 83.26%
